<a href="https://colab.research.google.com/github/Russo-Federico/DeepLearningFundamentals/blob/main/IBM-PyTorch-DL/0-PyTorchBasicsForDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a Neural Network

In [2]:
import torch
import torch.nn as nn
from torch import sigmoid

Create a NN class

In [3]:
class NN(nn.Module):
  # D_in - input size of the network
  # H - number of neurons
  # D_out - output size of the netowrk
  def __init__(self, D_in, H, D_out):
    super(NN, self).__init__()
    self.linear1 = nn.Linear(D_in, H) # input layer
    self.linear2 = nn.Linear(H, D_out) # output layer

  def forward(self, x):
    x = sigmoid(self.linear1(x))
    x = sigmoid(self.linear2(x))
    return x

In [4]:
x = torch.tensor([1.0])

model = NN(1,2,1)
yhat = model(x)
print("yhat:", yhat)

yhat: tensor([0.5798], grad_fn=<SigmoidBackward0>)


In [5]:
model.state_dict()

OrderedDict([('linear1.weight',
              tensor([[-0.8517],
                      [-0.9562]])),
             ('linear1.bias', tensor([ 0.0344, -0.0562])),
             ('linear2.weight', tensor([[-0.3093,  0.0581]])),
             ('linear2.bias', tensor([0.4012]))])

Build a network using the Sequential module

In [7]:
seq_model = torch.nn.Sequential(
    torch.nn.Linear(1,2),
    torch.nn.Sigmoid(),
    torch.nn.Linear(2,1),
    torch.nn.Sigmoid()
)

yhat = seq_model(x)
print("yhat:", yhat)

yhat: tensor([0.4922], grad_fn=<SigmoidBackward0>)


Train a model

In [8]:
def train(Y, X, model, optimizer, criterion, epochs=100):
  cost = []
  total = 0
  for epoch in range(epochs):
    total = 0
    for x, y in zip(X, Y):
      yhat = model(x)
      loss = criterion(yhat, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      total += loss.item()
    cost.append(total)
  return cost

In [11]:
model = NN(1,2,1)
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

X = torch.arange(-20,20,1).view(-1,1).type(torch.FloatTensor)
Y = torch.zeros(X.shape[0],1)
Y[(X[:,0] > -4) & (X[:,0] < 4)] = 1.0

cost = train(Y, X, model, optimizer, criterion)

print("cost: ", cost)

cost:  [27.439201295375824, 25.403576135635376, 23.88390800356865, 22.74214059114456, 21.878220349550247, 21.219715178012848, 20.713930368423462, 20.32234263420105, 20.016667306423187, 19.776038885116577, 19.584981471300125, 19.431942850351334, 19.30824166536331, 19.207310900092125, 19.124153167009354, 19.05494910478592, 18.996771708130836, 18.947377860546112, 18.905066192150116, 18.868557184934616, 18.836918339133263, 18.809488832950592, 18.78580704331398, 18.765535041689873, 18.748384058475494, 18.73404349386692, 18.722141474485397, 18.71226066350937, 18.70396415889263, 18.696845307946205, 18.690568044781685, 18.68487124145031, 18.679569974541664, 18.674537137150764, 18.669690057635307, 18.664968356490135, 18.660332411527634, 18.655751809477806, 18.651199460029602, 18.646653398871422, 18.642090663313866, 18.637491911649704, 18.63283821940422, 18.62811204791069, 18.623296856880188, 18.618378177285194, 18.613341212272644, 18.608176484704018, 18.602872863411903, 18.59742161631584, 18.59